In [ ]:
#%pip install -q --upgrade crewai langchain-openai python-dotenv openai

In [ ]:
1+1

In [ ]:
from crewai import Agent, Task, Crew, Process,LLM
from crewai.tools import tool
from openai import AzureOpenAI
import os
import requests
from datetime import datetime
import yaml
from dotenv
import pyodbc

In [ ]:
load_dotenv()

# Configuration - loaded from .env file
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")

# Validate configuration early for clearer errors
missing = [
    name for name, val in [
        ("AZURE_OPENAI_ENDPOINT", AZURE_OPENAI_ENDPOINT),
        ("AZURE_OPENAI_API_KEY", AZURE_OPENAI_API_KEY),
        ("AZURE_OPENAI_DEPLOYMENT", AZURE_OPENAI_DEPLOYMENT),
        ("AZURE_OPENAI_API_VERSION", AZURE_OPENAI_API_VERSION),
    ]
    if not val
]
if missing:
    raise ValueError(
        f"Missing required environment variables: {', '.join(missing)}.\n"
        "Create a .env file with these values or set them in your environment."
    )

# Configure CrewAI LLM for Azure OpenAI via OpenAI-compatible endpoint
llm = LLM(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_key=AZURE_OPENAI_API_KEY,
    base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT}",
    default_headers={"api-key": AZURE_OPENAI_API_KEY},
    default_query={"api-version": AZURE_OPENAI_API_VERSION},
    provider="openai",
)


In [ ]:


# Azure SQL connection
conn_str = (
    f"Driver={{ODBC Driver 18 for SQL Server}};"
    f"Server={os.getenv('AZURE_SQL_SERVER')};"
    f"Database={os.getenv('AZURE_SQL_DATABASE')};"
    f"Uid={os.getenv('AZURE_SQL_USER')};"
    f"Pwd={os.getenv('AZURE_SQL_PASSWORD')};"
    f"Encrypt=yes;TrustServerCertificate=no;"
)

def query_db(query):
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchall()

# Agent
analyst = Agent(
    role='IMDb Database Analyst',
    goal='Analyze IMDb movie data and provide insights',
    backstory='Expert SQL analyst specializing in movie data',
    llm=llm,
    verbose=True
)

# Task
task = Task(
    description="""
    Query the IMDb database to find:
    1. Top 10 highest rated movies
    2. Most prolific directors
    
    Use SQL: SELECT title, rating FROM movies ORDER BY rating DESC LIMIT 10
    """,
    agent=analyst,
    expected_output='Analysis report with top movies and directors'
)

# Run
crew = Crew(agents=[analyst], tasks=[task], verbose=True)
result = crew.kickoff()
print(result)